In [1]:
import os 
from dotenv import load_dotenv
load_dotenv()

True

Toools

In [2]:
from langchain_community.tools.tavily_search import TavilySearchResults

tools = [TavilySearchResults(max_results=3)]


Agent Execution 

In [3]:
import json
from json import JSONDecodeError
from typing import List, Union

from langchain_core.agents import AgentAction, AgentActionMessageLog, AgentFinish
from langchain_core.exceptions import OutputParserException
from langchain_core.messages import (
    AIMessage,
    BaseMessage,
)
from langchain_core.outputs import ChatGeneration, Generation

from langchain.agents.agent import AgentOutputParser


class OpenAIFunctionsAgentOutputParser(AgentOutputParser):
    """Parses a message into agent action/finish.

    Is meant to be used with OpenAI models, as it relies on the specific
    function_call parameter from OpenAI to convey what tools to use.

    If a function_call parameter is passed, then that is used to get
    the tool and tool input.

    If one is not passed, then the AIMessage is assumed to be the final output.
    """

    @property
    def _type(self) -> str:
        return "openai-functions-agent"

    @staticmethod
    def _parse_ai_message(message: BaseMessage) -> Union[AgentAction, AgentFinish]:
        """Parse an AI message."""
        if not isinstance(message, AIMessage):
            raise TypeError(f"Expected an AI message got {type(message)}")

        function_call = message.additional_kwargs.get("function_call", {})

        if function_call:
            function_name = function_call["name"]
            try:
                if len(function_call["arguments"].strip()) == 0:
                    # OpenAI returns an empty string for functions containing no args
                    _tool_input = {}
                else:
                    # otherwise it returns a json object
                    _tool_input = json.loads(function_call["arguments"], strict=False)
            except JSONDecodeError:
                raise OutputParserException(
                    f"Could not parse tool input: {function_call} because "
                    f"the `arguments` is not valid JSON."
                )

            # HACK HACK HACK:
            # The code that encodes tool input into Open AI uses a special variable
            # name called `__arg1` to handle old style tools that do not expose a
            # schema and expect a single string argument as an input.
            # We unpack the argument here if it exists.
            # Open AI does not support passing in a JSON array as an argument.
            if "__arg1" in _tool_input:
                tool_input = _tool_input["__arg1"]
            else:
                tool_input = _tool_input

            content_msg = f"responded: {message.content}\n" if message.content else "\n"
            log = f"\nInvoking: `{function_name}` with `{tool_input}`\n{content_msg}\n"
            return AgentActionMessageLog(
                tool=function_name,
                tool_input=tool_input,
                log=log,
                message_log=[message],
            )

        return AgentFinish(
            return_values={"output": message.content}, log=str(message.content)
        )

    def parse_result(
        self, result: List[Generation], *, partial: bool = False
    ) -> Union[AgentAction, AgentFinish]:
        if not isinstance(result[0], ChatGeneration):
            raise ValueError("This output parser only works on ChatGeneration output")
        message = result[0].message
        return self._parse_ai_message(message)

    def parse(self, text: str) -> Union[AgentAction, AgentFinish]:
        raise ValueError("Can only parse messages")


In [10]:
from langchain import hub
from langchain.agents import create_openai_functions_agent
from langchain_core.utils.function_calling import convert_to_openai_function
from langchain_core.runnables import Runnable, RunnablePassthrough
from langchain.agents.format_scratchpad.openai_functions import (
    format_to_openai_function_messages,
)

# from langchain.agents.output_parsers.openai_functions import (
#     OpenAIFunctionsAgentOutputParser,
# )
from langchain_community.chat_models import ChatOllama
from langchain_experimental.llms.ollama_functions import OllamaFunctions

# from langchain_openai import ChatOpenAI

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
# Choose the LLM that will drive the agent
# llm = ChatOpenAI(model="gpt-4-turbo-preview")

# llm = OllamaFunctions(model='mistral:7b-instruct-v0.2-q6_K', tool_system_prompt_template='')

llm = OllamaFunctions(model='mistral:7b-instruct-v0.2-q6_K')

# Construct the OpenAI Functions agent
agent_runnable = create_openai_functions_agent(llm, tools, prompt)

# llm_with_tools = llm.bind(functions=[convert_to_openai_function(t) for t in tools])

llm_with_tools = llm.bind(
    functions=[
    {'name': 'tavily_search_results_json', 
     'description': 'A search engine optimized for comprehensive, accurate, and trusted results. Useful for when you need to answer     questions about current events. Input should be a search query.', 
     'parameters': {'type': 'object', 
                    'properties': 
                        {'query': {'description': 'search query to look up', 'type': 'string'}}, 
                        'required': ['query']}}
    ]
)


agent = (
    RunnablePassthrough.assign(
        agent_scratchpad=lambda x: format_to_openai_function_messages(
            x["intermediate_steps"]
        )
    )
    | prompt
    | llm_with_tools
    | OpenAIFunctionsAgentOutputParser()
)

In [11]:
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_function_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')), MessagesPlaceholder(variable_name='chat_history',

In [12]:
from langgraph.prebuilt import create_agent_executor

agent_executor = create_agent_executor(agent, tools)




In [13]:
from langchain.globals import set_debug

set_debug(True)
from langchain.globals import set_verbose

set_verbose(True)

In [14]:
agent_executor.invoke(
    {"input": "who is the winnner of the us open", "chat_history": []}
)

[chain/start] [1:chain:LangGraph] Entering Chain run with input:
{
  "input": ""
}
[pregel/step] Starting step 0 with 1 task. Next tasks:
- __start__({'chat_history': [], 'input': 'who is the winnner of the us open'})
[chain/start] [1:chain:LangGraph > 2:chain:__start__] Entering Chain run with input:
{
  "input": "who is the winnner of the us open",
  "chat_history": []
}
[chain/end] [1:chain:LangGraph > 2:chain:__start__] s] Exiting Chain run with output:
{
  "output": "who is the winnner of the us open"
}
[chain/start] [1:chain:LangGraph > 3:chain:__start__] Entering Chain run with input:
{
  "input": "who is the winnner of the us open",
  "chat_history": []
}
[chain/end] [1:chain:LangGraph > 3:chain:__start__] s] Exiting Chain run with output:
{
  "output": []
}
[chain/start] [1:chain:LangGraph > 4:chain:__start__] Entering Chain run with input:
{
  "input": "who is the winnner of the us open",
  "chat_history": []
}
[chain/end] [1:chain:LangGraph > 4:chain:__start__] s] Exiting Ch

KeyError: 'tool'

# Plan and Execute - mar 19

In [1]:
from dotenv import load_dotenv
load_dotenv()

import os 
from langchain_community.llms import Ollama 
# from langchain_experimental.llms.ollama_functions import OllamaFunctions 

from langchain_community.tools.tavily_search import TavilySearchResults

tools = [TavilySearchResults(max_results=3)]



In [2]:
from langchain_core.language_models import BaseLanguageModel
from typing import Any, List, Optional, Sequence, Tuple, Type, Union
from langchain_core.prompts.chat import (
    BaseMessagePromptTemplate,
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
)
from langchain_core.tools import BaseTool
from langchain_core.utils.function_calling import convert_to_openai_function
from langchain.agents.format_scratchpad.openai_functions import (format_to_openai_function_messages)
from langchain.agents.output_parsers.openai_functions import (
    OpenAIFunctionsAgentOutputParser)
from langchain_core.runnables import Runnable, RunnablePassthrough


def create_openai_functions_agent(
    llm: BaseLanguageModel, tools: Sequence[BaseTool], prompt: ChatPromptTemplate
) -> Runnable:
   
    if "agent_scratchpad" not in prompt.input_variables:
        raise ValueError(
            "Prompt must have input variable `agent_scratchpad`, but wasn't found. "
            f"Found {prompt.input_variables} instead."
        )
        
    llm_with_tools = llm.bind(functions=[convert_to_openai_function(t) for t in tools])
    agent = (
        RunnablePassthrough.assign(
            agent_scratchpad=lambda x: format_to_openai_function_messages(
                x["intermediate_steps"]
            )
        )
        # | prompt
        | llm_with_tools
        # | OpenAIFunctionsAgentOutputParser()
    )
    return agent


In [3]:
prompt

NameError: name 'prompt' is not defined

In [4]:
llm_with_tools

NameError: name 'llm_with_tools' is not defined

In [5]:
import json
from typing import Any, Dict, List, Optional

from langchain_community.chat_models.ollama import ChatOllama
from langchain_core.callbacks import CallbackManagerForLLMRun
from langchain_core.language_models import BaseChatModel
from langchain_core.messages import AIMessage, BaseMessage
from langchain_core.outputs import ChatGeneration, ChatResult
from langchain_core.prompts import SystemMessagePromptTemplate

from langchain_experimental.pydantic_v1 import root_validator

DEFAULT_SYSTEM_TEMPLATE = """You have access to the following tools:

{tools}

You must always select one of the above tools and respond with only a JSON object matching the following schema:

{{
  "tool": <name of the selected tool>,
  "tool_input": <parameters for the selected tool, matching the tool's JSON schema>
}}
"""  # noqa: E501


DEFAULT_RESPONSE_FUNCTION = {
    "name": "__conversational_response",
    "description": (
        "Respond conversationally if no other tools should be called for a given query."
    ),
    "parameters": {
        "type": "object",
        "properties": {
            "response": {
                "type": "string",
                "description": "Conversational response to the user.",
            },
        },
        "required": ["response"],
    },
}


class OllamaFunctions(BaseChatModel):
    """Function chat model that uses Ollama API."""

    llm: ChatOllama

    tool_system_prompt_template: str

    @root_validator(pre=True)
    def validate_environment(cls, values: Dict) -> Dict:
        values["llm"] = values.get("llm") or ChatOllama(**values, format="json")
        values["tool_system_prompt_template"] = (
            values.get("tool_system_prompt_template") or DEFAULT_SYSTEM_TEMPLATE
        )
        return values

    @property
    def model(self) -> BaseChatModel:
        """For backwards compatibility."""
        return self.llm

    def _generate(
        self,
        messages: List[BaseMessage],
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
        **kwargs: Any,
    ) -> ChatResult:
        functions = kwargs.get("functions", [])
        if "function_call" in kwargs:
            functions = [
                fn for fn in functions if fn["name"] == kwargs["function_call"]["name"]
            ]
            if not functions:
                raise ValueError(
                    'If "function_call" is specified, you must also pass a matching \
function in "functions".'
                )
            del kwargs["function_call"]
        elif not functions:
            functions.append(DEFAULT_RESPONSE_FUNCTION)
        system_message_prompt_template = SystemMessagePromptTemplate.from_template(
            self.tool_system_prompt_template
        )
        system_message = system_message_prompt_template.format(
            tools=json.dumps(functions, indent=2)
        )
        if "functions" in kwargs:
            del kwargs["functions"]
        response_message = self.llm.predict_messages(
            [system_message] + messages, stop=stop, callbacks=run_manager, **kwargs
        )
        chat_generation_content = response_message.content
        if not isinstance(chat_generation_content, str):
            raise ValueError("OllamaFunctions does not support non-string output.")
        try:
            parsed_chat_result = json.loads(chat_generation_content)
        except json.JSONDecodeError:
            raise ValueError(
                f'"{self.llm.model}" did not respond with valid JSON. Please try again.'
            )
        called_tool_name = parsed_chat_result["tool"]
        called_tool_arguments = parsed_chat_result["tool_input"]
        called_tool = next(
            (fn for fn in functions if fn["name"] == called_tool_name), None
        )
        if called_tool is None:
            raise ValueError(
                f"Failed to parse a function call from {self.llm.model} \
output: {chat_generation_content}"
            )
        if called_tool["name"] == DEFAULT_RESPONSE_FUNCTION["name"]:
            return ChatResult(
                generations=[
                    ChatGeneration(
                        message=AIMessage(
                            content=called_tool_arguments["response"],
                        )
                    )
                ]
            )

        response_message_with_functions = AIMessage(
            content="",
            additional_kwargs={
                "function_call": {
                    "name": called_tool_name,
                    "arguments": json.dumps(called_tool_arguments)
                    if called_tool_arguments
                    else "",
                },
            },
        )

        return ChatResult(
            generations=[ChatGeneration(message=response_message_with_functions)]
        )

    @property
    def _llm_type(self) -> str:
        return "ollama_functions"


In [6]:
from langchain import hub

prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.pretty_print()

================================ System Message ================================

You are a helpful assistant

============================= Messages Placeholder =============================

{chat_history}

================================ Human Message =================================

{input}

============================= Messages Placeholder =============================

{agent_scratchpad}


In [7]:
from langchain import hub
# from langchain.agents import create_openai_functions_agent

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
# Choose the LLM that will drive the agent
# llm = ChatOpenAI(model="gpt-4-turbo-preview")
llm = OllamaFunctions(model='mistral:7b-instruct-v0.2-q6_K', tool_system_prompt_template='')

# Construct the OpenAI Functions agent

# agent_runnable = create_openai_functions_agent(llm, tools, prompt)

if "agent_scratchpad" not in prompt.input_variables:
    raise ValueError(
        "Prompt must have input variable `agent_scratchpad`, but wasn't found. "
        f"Found {prompt.input_variables} instead."
    )
    
llm_with_tools = llm.bind(functions=[convert_to_openai_function(t) for t in tools],
                          function_call={"name": "tavily_search_results_json"},)

agent = (
    RunnablePassthrough.assign(
        agent_scratchpad=lambda x: format_to_openai_function_messages(
            x["intermediate_steps"]
        )
    )
    | prompt
    | llm_with_tools
)

In [8]:
llm_with_tools

RunnableBinding(bound=OllamaFunctions(llm=ChatOllama(model='mistral:7b-instruct-v0.2-q6_K', format='json'), tool_system_prompt_template='You have access to the following tools:\n\n{tools}\n\nYou must always select one of the above tools and respond with only a JSON object matching the following schema:\n\n{{\n  "tool": <name of the selected tool>,\n  "tool_input": <parameters for the selected tool, matching the tool\'s JSON schema>\n}}\n'), kwargs={'functions': [{'name': 'tavily_search_results_json', 'description': 'A search engine optimized for comprehensive, accurate, and trusted results. Useful for when you need to answer questions about current events. Input should be a search query.', 'parameters': {'type': 'object', 'properties': {'query': {'description': 'search query to look up', 'type': 'string'}}, 'required': ['query']}}], 'function_call': {'name': 'tavily_search_results_json'}})

In [16]:
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_function_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')), MessagesPlaceholder(variable_name='chat_history',

In [17]:
# from langchain import hub
# # from langchain.agents import create_openai_functions_agent
# from langchain_openai import ChatOpenAI

# # Get the prompt to use - you can modify this!
# prompt = hub.pull("hwchase17/openai-functions-agent")
# # Choose the LLM that will drive the agent
# # llm = ChatOpenAI(model="gpt-4-turbo-preview")
# llm = OllamaFunctions(model='mistral:7b-instruct-v0.2-q6_K')

# # Construct the OpenAI Functions agent
# agent_runnable = create_openai_functions_agent(llm, tools, prompt)

In [18]:
from langchain.globals import set_debug
set_debug(True)
from langchain.globals import set_verbose
# set_verbose(True)

In [19]:
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_function_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')), MessagesPlaceholder(variable_name='chat_history',

In [20]:
from langgraph.prebuilt import create_agent_executor

agent_executor = create_agent_executor(agent, tools)

agent_executor.invoke(
    {"input": "who is the winnner of the us open", "chat_history": []}
)

[chain/start] [1:chain:LangGraph] Entering Chain run with input:
{
  "input": ""
}
[pregel/step] Starting step 0 with 1 task. Next tasks:
- __start__({'chat_history': [], 'input': 'who is the winnner of the us open'})
[chain/start] [1:chain:LangGraph > 2:chain:__start__] Entering Chain run with input:
{
  "input": "who is the winnner of the us open",
  "chat_history": []
}
[chain/end] [1:chain:LangGraph > 2:chain:__start__] s] Exiting Chain run with output:
{
  "output": "who is the winnner of the us open"
}
[chain/start] [1:chain:LangGraph > 3:chain:__start__] Entering Chain run with input:
{
  "input": "who is the winnner of the us open",
  "chat_history": []
}
[chain/end] [1:chain:LangGraph > 3:chain:__start__] s] Exiting Chain run with output:
{
  "output": []
}
[chain/start] [1:chain:LangGraph > 4:chain:__start__] Entering Chain run with input:
{
  "input": "who is the winnner of the us open",
  "chat_history": []
}
[chain/end] [1:chain:LangGraph > 4:chain:__start__] s] Exiting Ch

KeyError: 'tool'